In [1]:
# Simulation-Uhrzeit: Simulationszeit als Uhrzeit [hh:mm:ss] = simulation time
# Simulationssekunde: Simulationzeit
#Fahrzeugprotokol = Vehicle record
#Eindeutige Fahrzeugnummer = Vehicle number
#Entfernung auf der Strecke vom Beginn der Strecke oder Verbindungsstrecke = Position
#Querposition am Ende des Zeitschritts. Wertebereich 0-1:   = Position LAT
# 0: an der rechten Fahrstreifenkante
# 0.5: Mitte des Fahrstreifens
# 1: an der linken Fahrstreifenkante

# Beschleinigung = acceleration
# Geschwindigkeit = Speed

# Zeit im Netz : Die gesamte Zeit, die ein Fahrzeug im Netz ist:
# Der Wert wird erst in der letzten Sekunde, bevor das Fahrzeug das Netz verlässt, gespeichert.
# Protokollieren Sie in diesem Zeitschritt auch die Verlustzeit für das Fahrzeug.
# Für Fahrzeuge, die sich am Ende der Simulation noch im Netz befinden, können Sie die Gesamtzeit anhand ihrer Einsetzzeit ermitteln.

# https://datatofish.com/descriptive-statistics-pandas/
# https://stackoverflow.com/questions/10138085/how-to-plot-normal-distribution

# variance:
# https://blog.finxter.com/how-to-calculate-the-column-variance-of-a-dataframe-in-python-pandas/

#simulation second is from 0 to 7200.0
# steps are 0.5, 0.6, 0.7

# Vehicle type is 100, 150, 200, 300
# 100 = car, 150 = transporter, 200 = HGV, 300 = bus

In [2]:
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



In [3]:
# axis = 0 is Row, axis = 1 is Column
vehicle_routes_based = pd.read_csv("data/Trajectories/GothaerPlatz - baseline.csv")
len(vehicle_routes_based)

2443976

In [4]:
vehicle_routes_based_scenario1 = pd.read_csv("data/Trajectories/GothaerPlatz - Scenario 1.csv")
len(vehicle_routes_based_scenario1)

2413540

In [5]:
vehicle_routes_based.head()

,$VEHICLE:NO,SIMTMOFDAY,SIMSEC,POS,POSLAT,VEHTYPE,SPEED,ACCELERATION,DISTTRAVTOT,LENGTH,LANE\LINK\NO,LANE\INDEX
0,1,00:00:00.50,0.5,0.52,0.5,100,52.98,0.0,1.47,4.01,24,1
1,1,00:00:00.60,0.6,1.99,0.5,100,52.98,0.0,2.94,4.01,24,1
2,1,00:00:00.70,0.7,3.46,0.5,100,52.98,0.0,4.42,4.01,24,1
3,1,00:00:00.80,0.8,4.93,0.5,100,52.98,0.0,5.89,4.01,24,1
4,1,00:00:00.90,0.9,6.40,0.5,100,52.98,0.0,7.36,4.01,24,1


In [6]:
serie = vehicle_routes_based.iloc[2]

In [7]:
dic = serie.to_dict()

In [8]:
dic

{'$VEHICLE:NO': 1,
 'SIMTMOFDAY': '00:00:00.70',
 'SIMSEC': 0.7,
 'POS': 3.46,
 'POSLAT': 0.5,
 'VEHTYPE': 100,
 'SPEED': 52.98,
 'ACCELERATION': 0.0,
 'DISTTRAVTOT': 4.42,
 'LENGTH': 4.01,
 'LANE\\LINK\\NO': 24,
 'LANE\\INDEX': 1}

In [9]:
vehicle_routes_based.tail()

,$VEHICLE:NO,SIMTMOFDAY,SIMSEC,POS,POSLAT,VEHTYPE,SPEED,ACCELERATION,DISTTRAVTOT,LENGTH,LANE\LINK\NO,LANE\INDEX
2443971,4086,02:00:00.00,7200.0,18.61,0.5,100,37.03,-3.05,217.52,4.21,62,2
2443972,4087,02:00:00.00,7200.0,10.65,0.5,100,47.76,0.10,139.13,4.01,5,1
2443973,4088,02:00:00.00,7200.0,118.59,0.5,100,52.44,-0.04,119.64,4.61,24,1
2443974,4089,02:00:00.00,7200.0,108.56,0.5,100,51.89,0.00,109.54,4.64,24,2
2443975,4090,02:00:00.00,7200.0,94.67,0.5,100,57.13,0.00,94.84,4.21,77,1


In [10]:
vehicle_routes_based[(vehicle_routes_based["SIMSEC"] > 0.5) & (vehicle_routes_based["SIMSEC"] < 0.8)]

,$VEHICLE:NO,SIMTMOFDAY,SIMSEC,POS,POSLAT,VEHTYPE,SPEED,ACCELERATION,DISTTRAVTOT,LENGTH,LANE\LINK\NO,LANE\INDEX
1,1,00:00:00.60,0.6,1.99,0.5,100,52.98,0.0,2.94,4.01,24,1
2,1,00:00:00.70,0.7,3.46,0.5,100,52.98,0.0,4.42,4.01,24,1


In [11]:
vehicle_routes_based[["SIMSEC", "POS"]].describe()

,SIMSEC,POS
count,2.443976e+06,2.443976e+06
mean,3.352259e+03,6.726465e+01
std,2.018204e+03,6.773792e+01
min,5.000000e-01,0.000000e+00
25%,1.661600e+03,1.748000e+01
50%,3.169600e+03,4.442000e+01
75%,5.072700e+03,9.690000e+01
max,7.200000e+03,4.778900e+02


In [12]:
# show the type of cars
vehicle_routes_based["VEHTYPE"].describe()

count    2.443976e+06
mean     1.113693e+02
std      2.739654e+01
min      1.000000e+02
25%      1.000000e+02
50%      1.000000e+02
75%      1.000000e+02
max      3.000000e+02
Name: VEHTYPE, dtype: float64

In [13]:
vehicle_routes_based['VEHTYPE'].replace(to_replace=[100, 150, 200, 300], value=["Car", "Transporter", "HGV", "Bus"],
                                  inplace=True)


In [14]:
vehicle_routes_based

,$VEHICLE:NO,SIMTMOFDAY,SIMSEC,POS,POSLAT,VEHTYPE,SPEED,ACCELERATION,DISTTRAVTOT,LENGTH,LANE\LINK\NO,LANE\INDEX
0,1,00:00:00.50,0.5,0.52,0.5,Car,52.98,0.00,1.47,4.01,24,1
1,1,00:00:00.60,0.6,1.99,0.5,Car,52.98,0.00,2.94,4.01,24,1
2,1,00:00:00.70,0.7,3.46,0.5,Car,52.98,0.00,4.42,4.01,24,1
3,1,00:00:00.80,0.8,4.93,0.5,Car,52.98,0.00,5.89,4.01,24,1
4,1,00:00:00.90,0.9,6.40,0.5,Car,52.98,0.00,7.36,4.01,24,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2443971,4086,02:00:00.00,7200.0,18.61,0.5,Car,37.03,-3.05,217.52,4.21,62,2
2443972,4087,02:00:00.00,7200.0,10.65,0.5,Car,47.76,0.10,139.13,4.01,5,1
2443973,4088,02:00:00.00,7200.0,118.59,0.5,Car,52.44,-0.04,119.64,4.61,24,1
2443974,4089,02:00:00.00,7200.0,108.56,0.5,Car,51.89,0.00,109.54,4.64,24,2


In [15]:
vehicle_routes_based["VEHTYPE"].describe()

count     2443976
unique          4
top           Car
freq      2023937
Name: VEHTYPE, dtype: object

In [16]:
number_of_cars = vehicle_routes_based["$VEHICLE:NO"].unique()

In [17]:
number_of_cars_list_size = number_of_cars.size
number_of_cars_list_size

4090

In [18]:
vehicle1 = vehicle_routes_based[vehicle_routes_based["$VEHICLE:NO"] == 2000]

In [19]:
vehicle1

,$VEHICLE:NO,SIMTMOFDAY,SIMSEC,POS,POSLAT,VEHTYPE,SPEED,ACCELERATION,DISTTRAVTOT,LENGTH,LANE\LINK\NO,LANE\INDEX
1213713,2000,00:52:29.60,3149.6,0.23,0.5,Car,34.83,-0.04,0.97,4.21,61,1
1213758,2000,00:52:29.70,3149.7,1.20,0.5,Car,34.82,-0.04,1.94,4.21,61,1
1213803,2000,00:52:29.80,3149.8,2.16,0.5,Car,34.80,-0.04,2.90,4.21,61,1
1213848,2000,00:52:29.90,3149.9,3.13,0.5,Car,34.79,-0.04,3.87,4.21,61,1
1213893,2000,00:52:30.00,3150.0,4.10,0.5,Car,34.78,-0.04,4.83,4.21,61,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1237169,2000,00:53:35.30,3215.3,131.17,0.5,Car,55.81,0.23,846.99,4.21,31,2
1237200,2000,00:53:35.40,3215.4,132.72,0.5,Car,55.89,0.23,848.54,4.21,31,2
1237231,2000,00:53:35.50,3215.5,134.27,0.5,Car,55.97,0.23,850.09,4.21,31,2
1237262,2000,00:53:35.60,3215.6,135.83,0.5,Car,56.05,0.23,851.65,4.21,31,2


In [20]:
vehicle_routes_based[(vehicle_routes_based["SPEED"] < 10) & (vehicle_routes_based["ACCELERATION"] > 0)]

,$VEHICLE:NO,SIMTMOFDAY,SIMSEC,POS,POSLAT,VEHTYPE,SPEED,ACCELERATION,DISTTRAVTOT,LENGTH,LANE\LINK\NO,LANE\INDEX
5166,1,00:00:39.80,39.8,22.37,0.5,Car,7.33,0.33,545.32,4.01,74,2
5192,1,00:00:39.90,39.9,22.58,0.5,Car,7.63,0.83,545.52,4.01,74,2
5218,1,00:00:40.00,40.0,0.04,0.5,Car,8.07,1.22,545.74,4.01,128,1
5244,1,00:00:40.10,40.1,0.27,0.5,Car,8.62,1.54,545.97,4.01,128,1
5270,1,00:00:40.20,40.2,0.52,0.5,Car,9.27,1.79,546.22,4.01,128,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2438823,4068,01:59:40.90,7180.9,177.76,0.5,Car,5.11,2.61,178.19,4.01,24,2
2438851,4068,01:59:41.00,7181.0,177.91,0.5,Car,6.12,2.81,178.34,4.01,24,2
2438879,4068,01:59:41.10,7181.1,178.10,0.5,Car,7.15,2.85,178.53,4.01,24,2
2438907,4068,01:59:41.20,7181.2,178.31,0.5,Car,8.16,2.82,178.74,4.01,24,2


In [21]:
vehicle_routes_based[(vehicle_routes_based["SPEED"] < 1) & (vehicle_routes_based["ACCELERATION"] > 1)]


,$VEHICLE:NO,SIMTMOFDAY,SIMSEC,POS,POSLAT,VEHTYPE,SPEED,ACCELERATION,DISTTRAVTOT,LENGTH,LANE\LINK\NO,LANE\INDEX
9379,19,00:00:55.10,55.1,20.62,0.5,Car,0.97,1.25,319.63,4.01,54,1
24150,44,00:01:33.80,93.8,20.49,0.5,Car,0.59,1.05,318.90,4.61,54,1
26489,34,00:01:39.90,99.9,0.50,0.5,Car,0.83,1.21,545.39,4.21,128,1
33541,48,00:01:55.60,115.6,17.29,0.5,Car,0.94,1.22,462.25,4.21,87,1
36275,31,00:02:01.40,121.4,177.06,0.5,Car,0.62,1.10,178.38,4.61,24,2
...,...,...,...,...,...,...,...,...,...,...,...,...
2410978,4018,01:58:08.40,7088.4,20.53,0.5,Car,0.62,1.11,318.54,4.01,54,1
2424489,4033,01:58:52.20,7132.2,177.30,0.5,Car,0.64,1.15,177.45,4.64,24,2
2431953,4059,01:59:16.30,7156.3,20.69,0.5,Car,0.84,1.08,319.43,4.61,54,1
2433903,4057,01:59:22.80,7162.8,17.32,0.5,Car,0.95,1.23,462.07,4.64,87,1


In [22]:
vehicle_routes_based.groupby(["$VEHICLE:NO"]).mean()[["ACCELERATION","SPEED"]]

,ACCELERATION,SPEED
$VEHICLE:NO,,
1,-0.003030,48.137286
2,-0.007522,50.785543
3,-0.002965,36.107730
4,0.000137,54.407869
5,0.005890,53.149402
...,...,...
4086,-0.301597,54.327778
4087,-0.020000,47.697048
4088,0.074824,50.687412


In [23]:
## Scenario one
